In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import models,layers

In [ ]:
IMAGE_SIZE=256
BUFFER_SIZE=32

In [ ]:
tf.keras.preprocessing.image_dataset_from_directory("PlantVillage",)